In [1]:
import heapq
import itertools
import random
from dataclasses import dataclass
from collections import deque

In [2]:
@dataclass 
class Order:
    order_id: int
    side: str          # "buy" or "sell"
    price: float
    qty: int
    timestamp: int


@dataclass
class Trade:
    price: float
    qty: int
    buy_order_id: int
    sell_order_id: int
    timestamp: int

In [3]:
class OrderBook:
    def __init__(self):
        self.bids = {}  # price -> deque of orders
        self.asks = {}
        self.bid_heap = []  # max-heap via negative price
        self.ask_heap = []  # min-heap

    def add_order(self, order: Order):
        book = self.bids if order.side == "buy" else self.asks
        heap = self.bid_heap if order.side == "buy" else self.ask_heap
        price = order.price

        if price not in book:
            book[price] = deque()
            heapq.heappush(heap, -price if order.side == "buy" else price)

        book[price].append(order)

    def best_bid(self):
        while self.bid_heap:
            price = -self.bid_heap[0]
            if price in self.bids and self.bids[price]:
                return price
            heapq.heappop(self.bid_heap)
        return None

    def best_ask(self):
        while self.ask_heap:
            price = self.ask_heap[0]
            if price in self.asks and self.asks[price]:
                return price
            heapq.heappop(self.ask_heap)
        return None

    def match(self, timestamp):
        trades = []

        while True:
            bid_price = self.best_bid()
            ask_price = self.best_ask()

            if bid_price is None or ask_price is None:
                break
            if bid_price < ask_price:
                break

            buy_order = self.bids[bid_price][0]
            sell_order = self.asks[ask_price][0]

            trade_qty = min(buy_order.qty, sell_order.qty)
            trade_price = ask_price  # price-time priority

            trades.append(
                Trade(
                    price=trade_price,
                    qty=trade_qty,
                    buy_order_id=buy_order.order_id,
                    sell_order_id=sell_order.order_id,
                    timestamp=timestamp
                )
            )

            buy_order.qty -= trade_qty
            sell_order.qty -= trade_qty

            if buy_order.qty == 0:
                self.bids[bid_price].popleft()
            if sell_order.qty == 0:
                self.asks[ask_price].popleft()

        return trades

In [4]:
class Logger:
    def __init__(self):
        self.trades = []

    def log_trades(self, trades):
        self.trades.extend(trades)

In [5]:
class MarketEnvironment:
    def __init__(self, seed=42):
        self.order_book = OrderBook()
        self.logger = Logger()
        self.clock = 0
        random.seed(seed)
        self.order_id_gen = itertools.count()

    def submit_order(self, side, price, qty):
        order = Order(
            order_id=next(self.order_id_gen),
            side=side,
            price=price,
            qty=qty,
            timestamp=self.clock
        )
        self.order_book.add_order(order)

    def step(self):
        self.clock += 1
        trades = self.order_book.match(self.clock)
        self.logger.log_trades(trades)

    def observe(self):
        return {
            "best_bid": self.order_book.best_bid(),
            "best_ask": self.order_book.best_ask()
        }


In [6]:
class RandomTraderAgent:
    def act(self, env: MarketEnvironment):
        side = random.choice(["buy", "sell"])
        price = random.choice([99, 100, 101])
        qty = random.randint(1, 5)
        env.submit_order(side, price, qty)

In [7]:
if __name__ == "__main__":
    env = MarketEnvironment(seed=1)
    agent = RandomTraderAgent()

    for _ in range(20):
        agent.act(env)
        env.step()

    print("\nExecuted Trades:")
    if not env.logger.trades:
        print("No trades executed.")
    else:
        for trade in env.logger.trades:
            print(trade)
            
# Note: Not every step produces a trade.
# Trades occur only when best_bid >= best_ask.



Executed Trades:
Trade(price=99, qty=1, buy_order_id=0, sell_order_id=1, timestamp=2)
Trade(price=99, qty=3, buy_order_id=3, sell_order_id=1, timestamp=4)
Trade(price=100, qty=4, buy_order_id=4, sell_order_id=2, timestamp=5)
